# Milestone Project 1: Food Vision Big

## Check GPU

Google Colab offers free GPUs,all of them are comaptible for mixed precision training.. however,if you use other GPU not all of them are compatible with mixed precision training.

Google Colab offers:
* A100(compatible)
* V100(compatible)
* Tesla T4(compatible)
Knowing this, in order to use mixed precision training we need access to any one or if we're using our own hardware, our GPU needs a score of 7.0+(see here: https://developer.nvidia.com/cuda/cuda-gpus)


In [ ]:
!nvidia-smi -L

## Get helper functions


In [ ]:
# Download helper functions
!wget https://raw.githubusercontent.com//mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

In [ ]:
# Import series of helper functions for the notebook
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys

## Use TensorFlow Datasets to Download Data

In [ ]:
# Get TensorFlow Datasets
import tensorflow_datasets as tfds

In [ ]:
# list all available datasets
datasets_list = tfds.list_builders() # Get all available datasets in TFDS
print("food101" in datasets_list) # is our target datset in TFDS

In [ ]:
# Load in the data(takes 5-6 minutes in Colab)
(train_data, test_data), ds_info = tfds.load(name="food101",
                                             split=["train", "validation"],
                                             shuffle_files=False,
                                             as_supervised=True, #Data get returned in tuple format(data, label)
                                             with_info= True)

## Exploting the Food101 data from  TensorFlow Datasets
To become one with our data, we want to find:
* Class names
* The shape of our input data(image tensors)
* The datatype of our input data
* What the labels look like(e.g. are they one-hot encoded or they are label encoded)
* Do the labels match up with the class names?

In [ ]:
ds_info

In [ ]:
# Feature of Food101 from TFDS
ds_info.features

In [ ]:
# Get the class names
class_names = ds_info.features["label"].names
class_names[:10]

In [ ]:
# Take one sample of the train data
train_one_sample = train_data.take(1)  # samples are in format (image_tensor, label)

In [ ]:
# What does one sample of our training data look like?
train_one_sample

In [ ]:
# Output info about our training sample
for image, label in train_one_sample:
  print(f"""
  Image shape: {image.shape}
  Image datatype : {image.dtype}
  Target class from Food101(tensor form): {label}
  Class name (str form): {class_names[label.numpy()]}
  """)

In [ ]:

# What does our image tensor from TFDS's Food 101 look like?
image

In [ ]:
# What are the min and max values of image tensor?
import tensorflow as tf

tf.reduce_min(image), tf.reduce_max(image)

## Plot an image from TensorFlow Datasets

In [ ]:
# Plot an image tensor
import matplotlib.pyplot as plt
plt.imshow(image)
plt.title(class_names[label.numpy()])
plt.axis(False);

## Create preprocessing functions for our data
Neural networks perform best when data is in a certain way (e.g. batched, normalized, etc.)

However, not all data(including data from TensorFlow Datasets) comes like this.

So, in order to get it ready for a neural network, you'll often have to write preprocessing functions and map it to our data.

What we know about our data:
* In `uint8` datatype
* Comprised of all different size tensors (different sized images)
* Not scaled (the pixel values are between 0 and 255)

What we know model like:
* Data in `float32` dtype (or for mixed precision `float16` and `float32`)
* For batches, TensorFlow likes all of the tensors within a batch to be of the same size
* Scaled (values between 0 & 1 ) also called normalized tensors generally perform better

With these points in mind, we've got a few things we can tackle with a preprocessing function.

Since, we're going to be using an EfficientNetBX pretrained model from tf.keras.applications we don't need to rescale our data(these architectures have rescaling built-in)

This means our fucntions needs to:
1. Reshape our images to all the same size
2. Convert the dtype of our image tensors `uint8` to `float32`

In [ ]:
# Make a function for preprocessing images
def preprocess_img(image, label, img_shape=224):
  """
  Converts image datatype from 'uint8' -> 'float32' and reshapes
  image to [img_shape, img_shape, color_channels]
  """
  image = tf.image.resize(image, [img_shape, img_shape]) # reshape target image
  return tf.cast(image, tf.float32), label # return (float32_image, label)


In [ ]:
# Preprocess a single sample image and check the outputs
preprocessed_img = preprocess_img(image, label)[0]
print(f"Image before preprocessing:\n {image[:2]}..., \nShape: {image.shape}, \nDatatype: {image.dtype}")
print(f"Image after preprocessed: \n{preprocessed_img[:2]}...,\nShape:{preprocessed_img.shape}, \nDatatype:{preprocessed_img.dtype} ")

## Batch & prepare datasets

We'now going to make our data input pipeline run really fast

For more resources on this: https://www.tensorflow.org/guide/data_performance

In [ ]:
# Map preprocessing function to training(and parallelize)
train = train_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)

# Shuffle train_data and turn it into batches and prefetch it(load it faster)
train = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE) 

# Map preprocessing function to test data 
test = test_data.map(preprocess_img, num_parallel_calls=tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)


In [ ]:
train, test

>"Hey, TensorFlow, map this preprocessing function(`preprocess_img`) across the training dataset, then shuffle a number of elements and then batch them together and finally make sure to prepare new batches(prefetch) whilst the model is looking through(finding patterns) the current batch."

## Create modelling callbacks

We're going to create some of callbacks to help us while our model trains:
* TensorBoard callback to log training results (so we can visualize them later if needed to)
* ModelCheckPoint callback to save our model's progress after feature extraction

In [ ]:
# Create tensorboard (import from helper_function)
from helper_functions import create_tensorboard_callback

# Create a ModelCheckpoint callback to save a model's progress during training 
checkpoint_path = "model_checkpoints/cp.ckpt"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                     monitor="val_acc",
                                                     save_best_only=True,
                                                     save_weights_only=True,
                                                     verbose=0)

## Setup mixed precision training 

First and foremost, for a deeper understanding of mixed precision training, checkout training guide: https://www.tensorflow.org/guide/mixed_precision

Mixed precision utilizes a combination of float32 and float16 datatypes to speed up model performance

In [ ]:
# Turn ON mixed precision training 
from tensorflow.keras import mixed_precision

mixed_precision.set_global_policy("mixed_float16")#set global data policy to mixed precision 

In [ ]:
mixed_precision.global_policy()

## Build feature extraction model

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

# Create base model
input_shape = (224,224,3)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable= False

# Create functional model
inputs= layers.Input(shape=input_shape, name ="input_layer")
# Note: EfficientNetBX models have rescaling built-in but if model do not have it try belo:
# x= preprocessing.Rescaling(1/255.)(x)
x = base_model(inputs, training=False) #make sure layers which should be in inference mode only say like that
x= layers.GlobalAveragePooling2D(name="global_pool_layer")(x)
x= layers.Dense(len(class_names))(x)
outputs = layers.Activation("softmax", dtype=tf.float32, name="softmax_float32")(x)

model= tf.keras.Model(inputs, outputs)

# Compile the model
model.compile(loss="sparse_categorical_crossentropy",
             optimizer="adam",
             metrics=["accuracy"])

In [ ]:
model.summary()

## Checking layer dtype policies (are we using mixed precision?)

In [ ]:
# check the dtype_policy attributes of layers in our model
for layer in model.layers:
    print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

In [ ]:
# Check the dtype policy for base model
for  layer in base_model.layers[:20]:
    print( layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

## Fit the feature extraction model

In [ ]:
history_101_food_classes_feature_extract = model.fit(train_data,
                    steps_per_epoch=len(train_data),
                   validation_steps=int(0.15 * len(test_data)),
                   validation_data=(test_data),
                   epochs=3,
                   callbacks=[create_tensorboard_callback(dir_name="training_logs",
                                                         experiment_name="efficientnetb0_101_classes_all_data_feature_extract"),
                             model_checkpoint])

In [ ]:
# Evaluate model on whole data
results_feature_extractor = model.evaluate(test_data)

In [ ]:
plot_loss_curves(history_101_food_classes_feature_extract)

## Fine tuning the model
Fine-tuning the feature extraction model to beat the  [DeepFoodPaper](https://arxiv.org/pdf/1606.05675.pdf)

In [ ]:
# Download the saved model from Google Storage
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/07_efficientnetb0_feature_extract_model_mixed_precision.zip 

In [ ]:
from helper_functions import unzip_data
# Unzip the SavedModel downloaded from Google Stroage
!mkdir downloaded_gs_model # create new dir to store downloaded feature extraction model
!unzip 07_efficientnetb0_feature_extract_model_mixed_precision.zip -d downloaded_gs_model     

In [ ]:
# Loading feature extractor model
feature_extractor_model = tf.keras.models.load_model("/kaggle/working/downloaded_gs_model/07_efficientnetb0_feature_extract_model_mixed_precision")

In [ ]:
# How does the loaded model perform? (evaluate it on the test dataset)
loaded_model_results= feature_extractor_model.evaluate(test_data)

In [ ]:
# Get the summary of downloaded model
feature_extractor_model.summary()

In [ ]:
# Set all of the layers .trainable variable in the loaded model to True (so they're unfrozen)
for layer in feature_extractor_model.layers[1].layers:
    layer.trainable = True

In [ ]:
# Check all layers are trainable in feature extractor
for layer in feature_extractor_model.layers[1].layers:
    print(layer.name, layer.trainable)

In [ ]:
# Check to see what dtype_policy of the layers in your loaded model are
for layer in feature_extractor_model.layers:
    print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

In [ ]:
# Setup EarlyStopping callback to stop training if model's val_loss doesn't improve for 3 epochs
# Monitor the val_loss and stop training if it doesn't improve for 3 epochs
early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=3)

In [ ]:
# Create ModelCheckpoint callback to save best model during fine-tuning
# Save the best model only
# Monitor val_loss while training and save the best model (lowest val_loss)
fine_tune_checkpoint_path = "fine_tune_checkpoint/cp.ckpt"

fine_tune_checkpoint= tf.keras.callbacks.ModelCheckpoint(fine_tune_checkpoint_path, 
                                                        save_best_only=True,
                                                        monitor="val_loss",
                                                        save_weights_only=True)

In [ ]:
# Compile the model ready for fine-tuning
# Use the Adam optimizer with a 10x lower than default learning rate
feature_extractor_model.compile(loss="sparse_categorical_crossentropy",
                               optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.0001),
                               metrics=["accuracy"])

In [ ]:
# Start to fine-tune (all layers)..Use 100 epochs as the default..Validate on 15% of the test_data
# Use the create_tensorboard_callback, ModelCheckpoint and EarlyStopping callbacks you created eaelier
history_fine_tune_model = feature_extractor_model.fit(train_data, 
                                                     epochs=100,
                                                     steps_per_epoch=len(train_data),
                                                     validation_data=test_data,
                                                     validation_steps=int(0.15 * len(test_data)), 
                                                      callbacks=[fine_tune_checkpoint, early_stopping_callback,
                                                                create_tensorboard_callback(dir_name="training_logs",
                                                                                           experiment_name="fine_tune_101_food_class")])


In [ ]:
feature_extractor_model.save("fine_tuned model.h5")

## Evaluate your fine-tuned model

> Note: Due to limited RAM space(13GB) reloading the model 

In [ ]:
# Loading the fine tuned model
model_fine_tuned =  tf.keras.models.load_model("/kaggle/input/model-fine-tuned/fine_tuned model.h5")

In [ ]:
# Evaluate the model on test data
results_fine_tuned_model = model_fine_tuned.evaluate(test) 

## Making prediction with trained model

In [ ]:
# Make predictions with model
pred_prob = model_fine_tuned.predict(test)

In [ ]:
# How many predictions are there ?
len(pred_prob)

In [ ]:
# Check the shape of predictions
pred_prob.shape

In [ ]:
# Let's see what the first 5 predictions look like
pred_prob[:5]

In [ ]:
# what does the first prediction probability array look like ?
pred_prob[0] , len(pred_prob[0]), sum(pred_prob[0])

Our model outputs a prediction probability array(with N number of variables, where N is the number of classes) for each sample passed to the predict model.

In [ ]:
# We get onr prediction per class(101)
print(f"Number of prediction probabilities for sample 0 : {len(pred_prob[0])}")
print(f"What prediciton probability of sample 0 looks like: \n{pred_prob[0]}")
print(f"The class with highest predicted probability by the model for sample 0 is: {pred_prob[0].argmax()}" )

In [ ]:
# Get the predicted  classes of each label
pred_classes = pred_prob.argmax(axis=1)

# How do they look?
pred_classes[:10]

In [ ]:
# How many pred_classes we have?
len(pred_classes)

Now we've got a predictions array of all our model's predictions, to evaluate them , we need to compare them to original test dataset

In [ ]:
# To get the test labels we need to unravel our test_data PrefetchDataset
# labels = tf.expand_dims(labels, 0)
y_labels = []
for images, labels in test.unbatch():
    labels = tf.expand_dims(labels, 0)  # Expand dimensions of labels
    y_labels.append(labels[0].numpy())
y_labels[:10]

In [ ]:
len(y_labels)

## Evaluating the model's predictions 
One way to check that our model's predictions array is in the same order as our test labels array is to find the accuracy score

In [ ]:
results_fine_tuned_model

In [ ]:
# Let's try scikit-learn's accuracy score function and see what it comes up with
from sklearn.metrics import accuracy_score

sklearn_accuracy = accuracy_score(y_true=y_labels, y_pred=pred_classes)

In [ ]:
sklearn_accuracy

## Let's get visual: making a confusion matrix 

In [ ]:
from helper_functions import make_confusion_matrix

In [ ]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(10, 10), text_size=15, norm=False, savefig=False):
  """Makes a labelled confusion matrix comparing predictions and ground truth labels.

  If classes is passed, confusion matrix will be labelled, if not, integer class values
  will be used.

  Args:
    y_true: Array of truth labels (must be same shape as y_pred).
    y_pred: Array of predicted labels (must be same shape as y_true).
    classes: Array of class labels (e.g. string form). If `None`, integer labels are used.
    figsize: Size of output figure (default=(10, 10)).
    text_size: Size of output figure text (default=15).
    norm: normalize values or not (default=False).
    savefig: save confusion matrix to file (default=False).

  Returns:
    A labelled confusion matrix plot comparing y_true and y_pred.

  Example usage:
    make_confusion_matrix(y_true=test_labels, # ground truth test labels
                          y_pred=y_preds, # predicted labels
                          classes=class_names, # array of class label names
                          figsize=(15, 15),
                          text_size=10)
  """
  # Create the confustion matrix
  cm = confusion_matrix(y_true, y_pred)
  cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # normalize it
  n_classes = cm.shape[0] # find the number of classes we're dealing with

  # Plot the figure and make it pretty
  fig, ax = plt.subplots(figsize=figsize)
  cax = ax.matshow(cm, cmap=plt.cm.Blues) # colors will represent how 'correct' a class is, darker == better
  fig.colorbar(cax)

  # Are there a list of classes?
  if classes:
    labels = classes
  else:
    labels = np.arange(cm.shape[0])

  # Label the axes
  ax.set(title="Confusion Matrix",
         xlabel="Predicted label",
         ylabel="True label",
         xticks=np.arange(n_classes), # create enough axis slots for each class
         yticks=np.arange(n_classes),
         xticklabels=labels, # axes will labeled with class names (if they exist) or ints
         yticklabels=labels)

  # Make x-axis labels appear on bottom
  ax.xaxis.set_label_position("bottom")
  ax.xaxis.tick_bottom()

  ### Changed (plot x-labels vetically)###
  plt.xticks(rotation=70, fontsize=text_size)
  plt.yticks(fontsize=text_size)

  # Set the threshold for different colors
  threshold = (cm.max() + cm.min()) / 2.

  # Plot the text on each cell
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    if norm:
      plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)
    else:
      plt.text(j, i, f"{cm[i, j]}",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)

  # Save the figure to the current working directory
  if savefig:
    fig.savefig("confusion_matrix.png")

In [ ]:
make_confusion_matrix(y_true=y_labels,
                     y_pred=pred_classes,
                     classes=class_names,
                     figsize=(100,100),
                     text_size=20,
                     savefig=True)

## Let's keep the evaluation train going on, time for a classification report 

SciKit-learn has a helpful function fro acquiring many different classification metrics class(e.g. precision, recall and F1) called classification_report, let's try it out

In [ ]:
from sklearn.metrics import classification_report 
print(classification_report(y_true=y_labels,
                           y_pred=pred_classes))

In [ ]:
# Get a dictionary of the classification report 
classification_report_dict = classification_report(y_labels, pred_classes, output_dict=True)
classification_report_dict

Let's plot all F1-scores

In [ ]:
#Create a empty dictionary
class_f1_scores= {}

# Loop through classifiation report dictionary items
for key, value in classification_report_dict.items():
    if key == "accuracy": # stop once we get accuracy key
        break
    else:
        #Add class names and F1 scores to new dict
        class_f1_scores[class_names[int(key)]] = value["f1-score"]
class_f1_scores

In [ ]:
# Turn f1 scores into dataframe for visualization 
import pandas as pd
f1_scores = pd.DataFrame({"class_names": list(class_f1_scores.keys()),
                          "f1-score" : list(class_f1_scores.values())}).sort_values("f1-score", ascending=False)

In [ ]:
f1_scores

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12,25))

# PLot the horizontal bars
scores = ax.barh(range(len(f1_scores)), f1_scores["f1-score"].values)

# Set the y-ticks and labels
ax.set_yticks(range(len(f1_scores)))
ax.set_yticklabels(f1_scores["class_names"])

# Set the x-label and title
ax.set_xlabel("F1-score")
ax.set_title("F1-scores for 101 Different Food Classes(predicited by food vision model)")

# Invert the y-axis
ax.invert_yaxis()

# Add labels to the bars
for i , score in enumerate(f1_scores["f1-score"].values):
    ax.text(score, i, f"{score:.2f}", ha="left", va="center")

plt.show()

## Visualizing prediction on custom images
Now, this is the real test, how does our model go on food images not even in our test dataset(images of our own)

To visualize our model's predictions on our own images, we'll need a function to load and preprocess images, specifically it will need to:
* Resize the image to be in the same size as the images our model has trained on using tf.image.resize()
* Scale the image to get all of the pixel values between 0 & 1(if necessary)

In [ ]:
import tensorflow as tf
def load_and_prep_img(image, label, img_shape=224):
    """
    Converts image datatype from 'uint8' -> 'float32' and reshapes
    image to [img_shape, img_shape, color_channels]
    """
    image = tf.image.resize(image, [img_shape, img_shape])  # reshape target image
#     image = tf.cast(image, tf.int64)  # Convert to float32
    return image, label  # return (float32_image, label)


Now we've got a function to load and prepare target images, let's now write some code to visualize images, their target label predictions. Specifically, we'll write some code to:

1. Load a few random images from the test dataset
2. Make predictions on the loaded images
3. Plot the original image(s) along with the model's predictions, prediction probability and truth label

In [ ]:
import matplotlib.pyplot as plt
import random

def plot_random_images(test_data, model, class_labels, num_images=9, img_shape=224):
    """
    Plots a batch of random images from the test dataset and displays their true labels, predicted labels,
    and prediction probabilities.

    Parameters:
        test_data (tf.data.Dataset): The test dataset containing images and labels.
        model (tf.keras.Model): The trained model used for making predictions.
        class_labels (list): A list of class labels corresponding to the model's output classes.
        num_images (int): The number of random images to plot. Default is 9.
        img_shape (int): The desired shape of the images (img_shape x img_shape). Default is 224.

    Returns:
        None (plots the images and labels using matplotlib).
    """
    # Get a random batch of unique indices from the test dataset
    random_indices = random.sample(range(len(test_data)), num_images)

    # Initialize lists to store the images, true labels, and predicted labels
    batch_images = []
    true_labels = []
    predicted_labels = []

    # Fetch the images and labels for the random indices
    for i, (image, label) in enumerate(test_data):
        if i in random_indices:
            # Preprocess the image
            image, label = preprocess_img(image, label, img_shape)

            # Add the preprocessed image, true label, and corresponding index to the respective lists
            batch_images.append(image)
            true_labels.append(class_labels[label.numpy()])
            random_indices.remove(i)  # Remove the index from the list to ensure uniqueness

            # Make a prediction using the model
            prediction = model.predict(tf.expand_dims(image, axis=0))
            predicted_labels.append(class_labels[prediction.argmax()])

    # Plot the batch of images
    plt.figure(figsize=(17, 15))
    for i in range(num_images):
        image = batch_images[i]
        true_label = true_labels[i]
        predicted_label = predicted_labels[i]

        plt.subplot(3, 3, i+1)
        plt.imshow(image / 255.)

        if true_label == predicted_label:  # If predicted class matches true class, make text green
            title_color = "g"
        else:
            title_color = "r"
        plt.title(f"True Label: {true_label}, \n Predicted Label: {predicted_label},\n Prob: {prediction.max():.2f}", color=title_color)
        plt.axis(False)

    plt.show()


## Finding the most wrong Predictions

It's a good idea to go through at least 100+ random instances of your model's predictions to get a good feel for how it's doing.

After a while you might notice the model predicting on some images with a very high prediction probability, meaning it's very confident with its prediction but still getting the label wrong.

These most wrong predictions can help to give further insight into your model's performance.

So how about we write some code to collect all of the predictions where the model has output a high prediction probability for an image (e.g. 0.95+) but gotten the prediction wrong.

We'll go through the following steps:

1. Get all of the image file paths in the test dataset using the list_files() method.
2. Create a pandas DataFrame of the image filepaths, ground truth labels, prediction classes, max prediction probabilities, ground truth class names and predicted class names.

* **Note:** We don't necessarily have to create a DataFrame like this but it'll help us visualize things as we go.

3. Use our DataFrame to find all the wrong predictions (where the ground truth doesn't match the prediction).
4. Sort the DataFrame based on wrong predictions and highest max prediction probabilities.
5. Visualize the images with the highest prediction probabilities but have the wrong prediction.


In [ ]:
## TO do this is not working
# 1. Get the filenames of all of our test data
filepaths = []
for filepath in test_data.list_files("/root/tensorflow_datasets/food101/2.0.0", 
                                     shuffle=False):
  filepaths.append(filepath.numpy())
filepaths[:]

In [ ]:
# 2. Create a dataframe out of current prediction data for analysis
import pandas as pd
pred_df = pd.DataFrame({"y_true": y_labels,
                        "y_pred": pred_classes,
                        "pred_conf": pred_prob.max(axis=1), # get the maximum prediction probability value
                        "y_true_classname": [class_names[i] for i in y_labels],
                        "y_pred_classname": [class_names[i] for i in pred_classes]}) 
pred_df.head()

In [ ]:
# 3. Is the prediction correct?
pred_df["pred_correct"] = pred_df["y_true"] == pred_df["y_pred"]
pred_df.head()

And now since we know which predictions were right or wrong and along with their prediction probabilities, how about we get the 100 "most wrong" predictions by sorting for wrong predictions and descending prediction probabilties?

In [ ]:
# 4. Get the top 100 wrong examples
top_100_wrong = pred_df[pred_df["pred_correct"] == False].sort_values("pred_conf", ascending=False)[:100]
top_100_wrong.head(10)

Very interesting... just by comparing the ground truth classname (y_true_classname) and the prediction classname column (y_pred_classname), do you notice any trends?

It might be easier if we visualize them.

In [ ]:
## TO do